<CENTER><h1>Análisis del Bosón de Higgs del experimento ATLAS en el canal de decaimiento $H \rightarrow WW^{*}\rightarrow e\nu\mu\nu$     a      $\sqrt{s}=13TeV$</h1></CENTER>

-----------
<p>Vamos a incluir unas bibliotecas estándar de C++ y ROOT</p>

In [1]:
#include <iostream>
#include <string> 
#include <vector>
#include <stdio.h> 
#include <typeinfo>
using namespace std;

---------
<p>Vamos a incluir cabeceras y funciones creadas para proposito de este análisis</p>

In [2]:
#include "HWWAnalysisHistograms.h"
#include "make_HWW_objects.h"
#include "main_HWWAnalysis.C"
#include "array_sample.C"

----------
<h4>Which option should I run?</h4>
<p>Optioms are</p>
<p>0=run all data and Mc one after another</p>
<p>1=run data only</p>
<p>2=run MC sample only</p>


In [3]:
int option=1;
cout<<"option: "<<option<<endl;

option: 1


---------
<h4>Should use PROOF? (will make things faster)</h4>
<p>Options are:</p>
<p>0=NO</p>
<p>1=YES</p>

In [4]:
int parallel=0;
cout<<"Option: "<<parallel<<endl;

Option: 0


-----------
<h4>Selection of Events</h4>

In [5]:
vector<string> sample;
array_samples(sample,option); //function that is in charge of assigning the names of the analyzes to be studied
TString foption; //Replacement of option=GetOption(); ... 

-----------
<h3>Physical Analysis</h3>

In [6]:
//We define the histograms
define_histograms();


for (unsigned int i=0;i<sample.size();i++){ 
    
    fchain = main_HWWAnalysis(parallel,sample[i]);
    Init(fchain);
    Long64_t nentries = fReader.GetEntries(1);
    cout<< "Muestra a Analizar: "<< sample[i] << " # Events: " <<nentries << endl;
    
    while (fReader.Next()){
        
        //Scale factors
        Float_t scaleFactor = (*scaleFactor_ELE)*(*scaleFactor_MUON)*(*scaleFactor_LepTRIGGER)*(*scaleFactor_PILEUP);
        
        //MC weight
        Float_t m_mcWeight = *mcWeight;
        
        // read input option
        foption = sample[i]; //Replacement of option=GetOption();
        if (foption.Contains("single")){
            m_mcWeight = (*mcWeight/TMath::Abs(*mcWeight)); // set to 1 or -1 single top MCs
        }
        
        //Total Weight
        Float_t weight = scaleFactor*m_mcWeight;
        
        // Make difference between data y MC
        if (foption.Contains("data")) { weight = 1.; }
        
        //Preselection cut for electron/muon trigger
        if (*trigE || *trigM){
            
            // Preselection of good Leptons
            int goodlep_index[2];
            int goodlep_n = 0;
            int lep_index = 0;
            
            for (unsigned int j=0; j<*lep_n; j++){
                
                TLorentzVector leptemp;
                leptemp.SetPtEtaPhiE(lep_pt[j]/1000., lep_eta[j], lep_phi[j], lep_E[j]/1000.);
                
                // Lepton is Tight----boolean indicating whether lepton satisfies tight ID reconstruction criteria
                if (lep_isTightID->at(j)){
                    
                    // standard lepton isolation requirement => strict isolation
                    if (lep_pt[j]>15000. && (lep_ptcone30[j]/lep_pt[j]<0.1) && (lep_etcone20[j]/lep_pt[j]<0.1)){
                       
                        // electron selection
                        if (lep_type[j]==11 && TMath::Abs(lep_eta[j])<2.47 && (TMath::Abs(lep_eta[j])<1.37 || TMath::Abs(lep_eta[j])>1.52)){
                            if (TMath::Abs(lep_trackd0pvunbiased[j])/lep_tracksigd0pvunbiased[j]<5 && TMath::Abs(lep_z0[j]*TMath::Sin(leptemp.Theta()))<0.5){
                                goodlep_n = goodlep_n + 1;
                                goodlep_index[lep_index] = j;
                                lep_index++;
                            }
                        }
                        // muon selection
                        if (lep_type[j]==13 && TMath::Abs(lep_eta[j])<2.5){
                            if (TMath::Abs(lep_trackd0pvunbiased[j])/lep_tracksigd0pvunbiased[j]<3 && TMath::Abs(lep_z0[j]*TMath::Sin(leptemp.Theta()))<0.5){
                                goodlep_n = goodlep_n + 1;
                                goodlep_index[lep_index] = j;
                                lep_index++;
                            }
                        }
                    }
                }//End_Tight
                
            }
            
            //Exactly two good leptons, leading lepton with pT > 22 GeV and the subleading lepton with pT > 15 GeV
            if (goodlep_n==2){
                       
                int goodlep1_index = goodlep_index[0];
                int goodlep2_index = goodlep_index[1];
                
                if (lep_pt[goodlep1_index]>22000.){
                    
                    //two different-flavour opposite-sign leptons
                    if (lep_type[goodlep1_index] != lep_type[goodlep2_index]){
                        if (lep_charge[goodlep1_index]*lep_charge[goodlep2_index]<0){
                            
                            //TLorentzVector Definitions
                            TLorentzVector Lepton_1  = TLorentzVector();
                            TLorentzVector Lepton_2  = TLorentzVector();
                            TLorentzVector      MeT  = TLorentzVector();
  
                            Lepton_1.SetPtEtaPhiE(lep_pt[goodlep1_index]/1000., lep_eta[goodlep1_index], lep_phi[goodlep1_index],lep_E[goodlep1_index]/1000.);
                            Lepton_2.SetPtEtaPhiE(lep_pt[goodlep2_index]/1000., lep_eta[goodlep2_index], lep_phi[goodlep2_index],lep_E[goodlep2_index]/1000.);
                            MeT.SetPtEtaPhiE(*met_et/1000., 0, *met_phi , *met_et/1000.);
                            
                            TLorentzVector Lepton_12 = TLorentzVector();
                            Lepton_12 = Lepton_1 + Lepton_2;
                            
                            float mLL = Lepton_12.Mag(); //The invariant mass of the two leptons

                            float ptLL = Lepton_12.Pt(); //Transverse momentum of the dilepton system
                            
                            Float_t MET = *met_et/1000.; // Missing transverse momentum
                            
                            //Azimuthal angle between the two leptons
                            float dPhi_LL = TMath::Abs(lep_phi[goodlep1_index]-lep_phi[goodlep2_index]);
                            dPhi_LL = dPhi_LL < TMath::Pi() ? dPhi_LL : 2*TMath::Pi() - dPhi_LL;
                            
                            //Azimuthal angle between E_T^miss and the dilepton system
                            float dPhiLLmet = TMath::Abs(Lepton_12.Phi() - MeT.Phi());
                            dPhiLLmet    = dPhiLLmet < TMath::Pi() ? dPhiLLmet : 2*TMath::Pi() - dPhiLLmet;
                            
                            //dilepton transverse mass
                            float mt = sqrt(2*Lepton_12.Pt()*MeT.Et()*(1-cos(Lepton_12.DeltaPhi(MeT))));
                            
                            // Preselection of good jets
                            int goodjet_n = 0;
                            int goodbjet_n = 0;
                            
                            int goodjet_index[*jet_n];
                            int jet_index = 0;
                            
                            int goodbjet_index[*jet_n];
                            int bjet_index = 0;
                            
                            for (unsigned int j=0; j<*jet_n; j++){
                                
                                if (jet_pt[j]>20000. && TMath::Abs(jet_eta[j])<2.5){
                                    
                                    //JVT cleaning
                                    bool jvt_pass = true;
                                    
                                    if (jet_pt[j]<60000. && TMath::Abs(jet_eta[j])<2.4 && jet_jvt[j]<0.59) {jvt_pass=false;}
                                    
                                    if (jvt_pass){
                                        
                                        // cut on 85% WP
                                        if (jet_MV2c10[j] > 0.1758475){
                                            goodbjet_n++;
                                            goodbjet_index[bjet_index] = j;
                                            bjet_index++;
                                        }
                                        
                                        if (jet_pt[j]>30000.){
                                            goodjet_n++;
                                            goodjet_index[jet_index] = j;
                                            jet_index++;
                                        }
                                    }
                                    
                                }
                            }
                            
                            // fill histogram before any cut
                            double names_of_global_variable_pre[]={mLL, ptLL, dPhi_LL, dPhiLLmet, MET, mt, (double)goodjet_n,
                                                                   (double)goodbjet_n};
                            TString histonames_of_global_variable_pre[]={"histI_mLL", "histI_ptLL", "histI_dPhi_LL", "histI_dPhiLLmet",
                                                                         "histI_etmiss", "histI_mt", "histI_n_jets", "histI_n_bjets"};
                            int length_global_pre = arraysize(names_of_global_variable_pre);
                            for (unsigned int j=0; j<length_global_pre; j++ ){
                                FillHistogramsGlobal(names_of_global_variable_pre[j], weight , histonames_of_global_variable_pre[j]);
                            }
                           
                            
                            //  remove low mass meson resonances and DY events; ggF regions, at least 1 jet
                            if ( mLL > 10 && goodjet_n <= 1 && MET > 30){
                                
                                if (mLL < 55){
                                    
                                    if (ptLL > 30){
                                        
                                        if (dPhi_LL < 1.8){
                                            
                                            if (dPhiLLmet > TMath::Pi()/2){
                                                
                                                if (goodbjet_n ==0){
                                                    
                                                    
                                                    //Start to fill histograms : definitions of x-axis variables
                                                    
                                                    double names_of_global_variable[]={mLL, ptLL, dPhi_LL, dPhiLLmet, MET, mt, (double)goodjet_n,
                                                                                       (double)goodbjet_n};
                                                    double names_of_leadlep_variable[]={Lepton_1.Pt(), Lepton_1.Eta(), Lepton_1.E(), Lepton_1.Phi(),
                                                                                        (double)lep_charge[goodlep1_index], (double)lep_type[goodlep1_index]};
                                                    double names_of_subleadlep_variable[]={Lepton_2.Pt(), Lepton_2.Eta(), Lepton_2.E(), Lepton_2.Phi(),
                                                                                           (double)lep_charge[goodlep2_index], (double)lep_type[goodlep2_index]};
                                                    
                                                    //Start to fill histograms : definitions of histogram names
                                                    
                                                    TString histonames_of_global_variable[]={"hist_mLL", "hist_ptLL", "hist_dPhi_LL", "hist_dPhiLLmet",
                                                                                             "hist_etmiss", "hist_mt", "hist_n_jets", "hist_n_bjets"};
                                                    TString histonames_of_leadlep_variable[]={"hist_leadleptpt", "hist_leadlepteta", "hist_leadleptE",
                                                                                              "hist_leadleptphi", "hist_leadleptch", "hist_leadleptID"};
                                                    TString histonames_of_subleadlep_variable[]={"hist_subleadleptpt", "hist_subleadlepteta", "hist_subleadleptE",
                                                                                                 "hist_subleadleptphi", "hist_subleadleptch", "hist_subleadleptID"};
                                                    int length_global = arraysize(names_of_global_variable);
                                                    int length_leadlep = arraysize(names_of_leadlep_variable);
                                                    int length_subleadlep = arraysize(names_of_subleadlep_variable);
                                                    
                                                    // Fill histograms
                                                    
                                                    for (unsigned int j=0; j<length_global; j++){
                                                        FillHistogramsGlobal(names_of_global_variable[j], weight , histonames_of_global_variable[j]);
                                                    }
                                                    
                                                    for (unsigned int j=0; j<length_leadlep; j++){
                                                        FillHistogramsLeadlept(names_of_leadlep_variable[j], weight , histonames_of_leadlep_variable[j]);
                                                    }
                                                    
                                                    for (unsigned int j=0; j<length_subleadlep; j++){
                                                        FillHistogramsSubleadlept(names_of_subleadlep_variable[j], weight, histonames_of_subleadlep_variable[j]);
                                                    }
                                                    
                                                }
                                            }
                                        }
                                    }
                                }
                            }
                        }
                    }
                }
            }
        }
    }
    
    // Analysis results are saved
    Terminate(sample[i]);
    
    //reset the histograms
    reset_histograms();
    
    // Reset fchain
    fchain->Reset();
}

Warning in <TClass::Init>: no dictionary for class ROOT::TIOFeatures is available


Muestra a Analizar: data_A # Events: 668152
Writting with name option: data_A
Muestra a Analizar: data_B # Events: 2459370
Writting with name option: data_B
Muestra a Analizar: data_C # Events: 3587872
Writting with name option: data_C
Muestra a Analizar: data_D # Events: 5490396
Writting with name option: data_D
